<a href="https://colab.research.google.com/github/akingulpinar/CNG562_Otto_Product_Classification/blob/master/Gulpinar_Otto_Product_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
import os

In [0]:
os.chdir("C:\\Users\\admin\\Desktop\\Akın\\CNG562Assignment2\\1\\")
"""os.chdir("C:\\Users\\aking\\OneDrive - metu.edu.tr\\LECTURES\\8th Semester\\CNG562\\Homework2")"""

dataset = pd.read_csv("train.csv")
dataset = dataset.drop('id',axis=1)

X= pd.get_dummies(dataset.drop('target',axis=1),drop_first=True).iloc[:,:].values
Y= dataset.iloc[:,93].values

In [0]:
from sklearn.preprocessing import LabelEncoder
labelEncoder_Y = LabelEncoder()
Y = labelEncoder_Y.fit_transform(Y)

In [0]:
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(Y)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25)

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
#Artificial Neural Networks
import keras
from keras.models import Sequential
from keras.layers import Dense

def keras_model():
    classifier = Sequential()
    classifier.add(Dense(51,kernel_initializer='uniform', activation='relu', input_dim=93))
    classifier.add(Dense(9,kernel_initializer='uniform', activation='softmax'))
    classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return classifier

classifier = keras_model()
print(classifier.summary())

classifier.fit(X_train, y_train, epochs=50, batch_size=500)
y_pred= classifier.predict(X_test)

from keras.wrappers.scikit_learn import KerasClassifier
estimator = KerasClassifier(build_fn=keras_model, epochs=100, batch_size=500)

from sklearn.model_selection import cross_val_score
results = cross_val_score(estimator, X, dummy_y, cv=10)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
#Support Vector Machines
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix

classifierSVM = SVC(kernel="rbf", gamma="auto")
classifierSVM.fit(X_train,y_train)
y_predictSVM = classifierSVM.predict(X_test)

cmSVM =confusion_matrix(y_test,y_predictSVM)
accuracy_score(y_test,y_predictSVM)

accuraciesSVM = cross_val_score(estimator= classifierSVM,X=X_train,y=Y_train,cv=10)
accuraciesSVM.mean()

In [0]:
#K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier 

classifierKNN = KNeighborsClassifier(n_neighbors=17, metric="minkowski", p=2)
classifierKNN.fit(X_train,y_train)
y_predictKNN = classifierKNN.predict(X_test)

cmKNN =confusion_matrix(y_test,y_predictKNN)
accuracy_score(y_test,y_predictKNN)

In [0]:
#MLPClassifier
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(solver= 'lbfgs', hidden_layer_sizes= (30, 10), alpha= 1e-5)
classifier.fit(X_train,y_train)
y_pred= classifier.predict(X_test)


cmSVM =confusion_matrix(y_test,y_pred)
accuracy_score(y_test,y_pred)

In [0]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

classifierRF = RandomForestClassifier(n_estimators=100,max_depth=2)
classifierRF.fit(X_train,Y_train)

Y_predRFC = classifierRF.predict(X_test)
cmClassifierRF = confusion_matrix(Y_test,Y_predRFC)
accuracy_score(Y_test,Y_predRFC)

In [0]:
#XGBoost
from xgboost import XGBClassifier

classifierXGB = XGBClassifier(max_depth=3, n_estimators=1000,learning_rate=0.01,subsample=0.8,colsample_bytree=1, gamma=1)
classifierXGB.fit(X_train,Y_train)

Y_predict = classifierXGB.predict(X_test)
cmXGB =confusion_matrix(Y_test,Y_predict)
accuracy_score(Y_test,Y_predict)